In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import BuildVersion

import re
import pandas as pd


def get_suffix(x):
    suffix = x.split(".", 1)[-1]
    return re.sub(r"\.\d+", "", suffix)


results = {}

all_suffixes = ["isra", "part", "cold", "constprop", "localalias"]

for v in BuildVersion.filter():
    df = v.img.symtab.funcs_renamed

    num_orig = 0
    num_orig_local = 0
    num_orig_nonlocal = 0
    all_funcs = v.img.symtab.funcs
    all_names = {row["name"]: row["bind"] for _, row in all_funcs.iterrows()}

    for name in df["name"]:
        if ".cold" not in name:
            continue
        stem = name.split(".", 1)[0]
        if stem in all_names:
            bind = all_names[stem]
            num_orig += 1
            if bind == "STB_LOCAL":
                num_orig_local += 1
            else:
                num_orig_nonlocal += 1
            # print(name, stem, all_funcs[all_funcs["name"] == stem])

    df = df["name"].map(get_suffix).value_counts().to_frame()

    for suffix in all_suffixes:
        if suffix not in df.index:
            df.loc[suffix] = 0

    multiple = df.index.str.contains(r"\.")
    df = pd.concat([df[~multiple].T, df[multiple].sum().to_frame("≥2")], axis=1)

    df["renamed"] = df.sum(axis=1)
    df["num_orig"] = num_orig
    df["num_orig_local"] = num_orig_local
    df["num_orig_nonlocal"] = num_orig_nonlocal

    df["funcs"] = len(v.img.symtab.funcs)
    df["%"] = df["renamed"] / df["funcs"]
    df["local"] = len(v.img.symtab.funcs_local)
    df["nonlocal"] = len(v.img.symtab.funcs_nonlocal)
    df["gcc"] = v.img.gcc_version

    results[v] = df


df = pd.concat(results).fillna(0)

df.index = df.index.droplevel(-1)

df

[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.4.0-21-generic-amd64.pkl
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.8.0-22-generic-amd64.pkl
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.10.0-19-generic-amd64.pkl
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.13.0-16-generic-amd64.pkl
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.15.0-20-generic-amd64.pkl
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.18.0-10-generic-amd64.pkl
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.0.0-13-generic-amd64.pkl
[   symtab.py:37 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.3.0-18-generic-amd64.pkl
[   symtab.py:37 ] I

,isra,part,constprop,localalias,cold,≥2,renamed,num_orig,num_orig_local,num_orig_nonlocal,funcs,%,local,nonlocal,gcc
4.4.0-21-generic-amd64,577,522,198,1,0,64,1362,0,0,0,38939,0.034978,21254,17685,5.3.1
4.8.0-22-generic-amd64,578,561,234,0,0,63,1436,0,0,0,40839,0.035162,22234,18605,6.2.0
4.10.0-19-generic-amd64,589,575,251,0,0,59,1474,0,0,0,41828,0.035240,22763,19065,6.3.0
4.13.0-16-generic-amd64,608,725,253,0,0,70,1656,0,0,0,43640,0.037947,23818,19822,7.2.0
4.15.0-20-generic-amd64,628,760,258,0,0,65,1711,0,0,0,45304,0.037767,24681,20623,7.3.0
4.18.0-10-generic-amd64,709,736,263,0,1875,161,3744,1893,923,970,49095,0.076260,27751,21344,8.2.0
5.0.0-13-generic-amd64,721,749,262,0,1910,157,3799,1926,952,974,49933,0.076082,28139,21794,8.3.0
5.3.0-18-generic-amd64,785,774,428,0,4357,334,6678,4397,2185,2212,54601,0.122305,32110,22491,9.2.1
5.4.0-26-generic-amd64,783,774,409,0,3228,247,5441,3256,1737,1519,54333,0.100142,31504,22829,9.3.0
5.8.0-25-generic-amd64,612,974,764,0,3628,372,6350,3717,2174,1543,59992,0.105847,36352,23640,10.2.0


In [3]:
from depsurf.plot import plot_yticks, save_fig, bold

import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(8, 3))

colors = {
    "isra": "C0",
    "part": "C1",
    "cold": "C2",
    "constprop": "C3",
    "≥2": "C4",
}

xs = np.arange(len(df.index))
bottom = np.zeros(len(df.index))
for col in list(colors):
    ax.bar(xs, df[col], bottom=bottom, label=col, color=colors[col])
    bottom += df[col]

for x, b, p in zip(xs, bottom, df["%"]):
    ax.text(x, b, f"{p:.0%}", ha="center", va="bottom", fontsize=8)

labels = [bold(v.short_version) if v.lts else v.short_version  for v in df.index]
labels = [f"{l}\n{gcc[:gcc.rfind('.')]}" for l, gcc in zip(labels, df["gcc"])]
ax.set_xticks(xs, labels, rotation=0)

plot_yticks(ax)

ax.set_ylabel("Number of renamed functions")
ax.set_xlabel("Linux kernel version and GCC version")
ax.legend()

ax.tick_params(labelsize=9)

save_fig(fig, "rename")

[      mpl.py:57 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/rename.pdf


4.15 -> 4.18:
- `-freorder-blocks-and-partition` enabed by default in GCC 8

- https://gcc.gnu.org/gcc-8/changes.html

5.0 -> 5.3:
- "Hot/cold partitioning is now more precise and aggressive." since GCC 9

- https://gcc.gnu.org/gcc-9/changes.html

5.13 -> 5.15:

- Non-static functions renamed e.g., `ipv6_local_rxpmtu`

5.19 -> 6.2: 

- `-falign-functions=16` added since Linux 6.2

- https://github.com/torvalds/linux/commit/c27cd083cfb9d392f304657ed00fcde1136704e7

- https://github.com/torvalds/linux/commit/d49a0626216b95cd4bf696f6acf55f39a16ab0bb

In [4]:
def get_flags(version):
    return set(BuildVersion.filter(version=version)[0].img.dwarf.flags)


get_flags("5.15") - get_flags("5.13")

{'-fsanitize=bool',
 '-fsanitize=bounds',
 '-fsanitize=enum',
 '-fsanitize=shift',
 '-gdwarf-5'}

In [5]:
def get_cold_funcs(version):
    df = BuildVersion.filter(version=version)[0].img.symtab.funcs_renamed
    # return df
    names = df["name"]
    return set(names[names.str.contains(".cold")])


# get_cold_funcs("5.15") - get_cold_funcs("5.13")

# df = get_cold_funcs("5.15")

# df